<a href="https://colab.research.google.com/github/Annie-Yeeun-Jang/Info_retrieval_project/blob/master/AE_keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import json
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.sparse.linalg import svds

from google.colab import drive
drive.mount('/content/gdrive')

import os
os.chdir('/content/gdrive/MyDrive/650/team_proj')

Mounted at /content/gdrive


In [2]:
import time
import torch 
import torch.nn as nn
import torch.optim as optim 
import torch.nn.functional as F

In [3]:
#jester=pd.read_csv('jester.csv')
gt_matrix=pd.read_pickle('gt_matrix.pkl')
train_matrix=pd.read_pickle('masked_for_train.pkl')

In [4]:
gt_matrix2=gt_matrix.fillna(-30).drop(['test_joke_idx'], axis=1)
zero_one_gt=pd.DataFrame((gt_matrix2.values > 0).astype(int), index=gt_matrix2.index, columns=gt_matrix2.columns)
zero_one_gt

,5,7,8,13,15,16,17,18,19,20,...,141,142,143,144,145,146,147,148,149,150
0,1,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,1,1,1,1,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,1,0,0,0,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,1,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,1,1,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44796,0,1,0,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
44797,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
44798,0,1,1,1,1,1,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
44799,0,0,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
zero_one_train=pd.DataFrame((train_matrix.values > 0).astype(int), index=train_matrix.index, columns=train_matrix.columns)
zero_one_train

,5,7,8,13,15,16,17,18,19,20,...,141,142,143,144,145,146,147,148,149,150
0,1,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,1,1,1,1,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,1,0,0,0,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,1,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,1,1,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44796,0,0,0,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
44797,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
44798,0,1,1,1,1,0,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
44799,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
zero_one_train.shape

(44801, 140)

In [7]:
n_jokes=140

In [8]:
#from https://blog.keras.io/building-autoencoders-in-keras.html
import keras
from keras import layers

# This is the size of our encoded representations
encoding_dim = 32  # 32 floats -> compression of factor 24.5, assuming the input is 784 floats

# This is our input image
input_img = keras.Input(shape=(140,))
# "encoded" is the encoded representation of the input
encoded = layers.Dense(encoding_dim, activation='relu')(input_img)
# "decoded" is the lossy reconstruction of the input
decoded = layers.Dense(140, activation='sigmoid')(encoded)

# This model maps an input to its reconstruction
autoencoder = keras.Model(input_img, decoded)

In [9]:
encoder = keras.Model(input_img, encoded)
# This is our encoded (32-dimensional) input
encoded_input = keras.Input(shape=(encoding_dim,))
# Retrieve the last layer of the autoencoder model
decoder_layer = autoencoder.layers[-1]
# Create the decoder model
decoder = keras.Model(encoded_input, decoder_layer(encoded_input))

In [13]:
autoencoder.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 140)]             0         
                                                                 
 dense (Dense)               (None, 32)                4512      
                                                                 
 dense_1 (Dense)             (None, 140)               4620      
                                                                 
Total params: 9,132
Trainable params: 9,132
Non-trainable params: 0
_________________________________________________________________


In [10]:
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

In [ ]:
from keras.datasets import mnist
import numpy as np
(x_train, _), (x_test, _) = mnist.load_data()

11490434/11490434 [==============================] - 0s 0us/step


In [ ]:
x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.
x_train = x_train.reshape((len(x_train), np.prod(x_train.shape[1:])))
x_test = x_test.reshape((len(x_test), np.prod(x_test.shape[1:])))
print(x_train.shape)
print(x_test.shape)

(60000, 784)
(10000, 784)


In [11]:
x_train=zero_one_train[:-4000]
x_test=zero_one_train[-4000:]
gt_train=zero_one_gt[:-4000]
gt_test=zero_one_gt[-4000:]

In [12]:
autoencoder.fit(x_train, gt_train,
                epochs=50,
                batch_size=256,
                shuffle=True,
                validation_data=(x_test,gt_test))

Epoch 1/30
160/160 [==============================] - 2s 6ms/step - loss: 0.4518 - val_loss: 0.2539
Epoch 2/30
160/160 [==============================] - 1s 5ms/step - loss: 0.2705 - val_loss: 0.2013
Epoch 3/30
160/160 [==============================] - 1s 4ms/step - loss: 0.2320 - val_loss: 0.1738
Epoch 4/30
160/160 [==============================] - 1s 5ms/step - loss: 0.2080 - val_loss: 0.1549
Epoch 5/30
160/160 [==============================] - 1s 4ms/step - loss: 0.1897 - val_loss: 0.1402
Epoch 6/30
160/160 [==============================] - 1s 4ms/step - loss: 0.1747 - val_loss: 0.1287
Epoch 7/30
160/160 [==============================] - 1s 4ms/step - loss: 0.1626 - val_loss: 0.1198
Epoch 8/30
160/160 [==============================] - 1s 4ms/step - loss: 0.1532 - val_loss: 0.1134
Epoch 9/30
160/160 [==============================] - 1s 4ms/step - loss: 0.1460 - val_loss: 0.1084
Epoch 10/30
160/160 [==============================] - 1s 4ms/step - loss: 0.1406 - val_loss: 0.1046

test는 그냥 마지막 10% users 떼어내서 할까봄

In [17]:
encoded_imgs = encoder.predict(x_test)
restored = decoder.predict(encoded_imgs) #decoded images = final prediction for test data

125/125 [==============================] - 0s 1ms/step


In [18]:
restored.shape

(4000, 140)

In [27]:
restored=pd.DataFrame(restored, columns=x_test.columns, index=x_test.index )

In [19]:
from sklearn.metrics import ndcg_score
def get_rank(arr):
  return arr.argsort()[::-1].argsort()+1

In [21]:
x_test.index

RangeIndex(start=40801, stop=44801, step=1)

In [50]:
ndcg_cum=0
for user_id in x_test.index:
  gt_rank=get_rank(x_test.loc[user_id])
  pred_rank=get_rank(restored.loc[user_id])
  ndcg_cum+=ndcg_score(np.asarray([gt_rank]), np.asarray([pred_rank]), k=10)

In [51]:
ndcg_total=ndcg_cum/len(x_test)
ndcg_total #for k=5

0.5296730653756404

In [52]:
ndcg_total=ndcg_cum/len(x_test)
ndcg_total #for k=10

0.5296730653756404

In [48]:
hit=0
k=10
for user_id in x_test.index:
  top_k_pred=list(restored.loc[user_id].sort_values(ascending=False).index[:k])
  #top_true=zero_one_gt.loc[user_id].sort|_values(ascending=False).index[0] #이부분 코드 수정 !! (row별로 test idx 뽑아와서 소팅 없이)
  top_true=gt_matrix.loc[user_id,'test_joke_idx']
  if top_true in top_k_pred:
    hit+=1

In [47]:
hit_ratio=hit/len(x_test)
hit_ratio #for k=5

0.02925

In [49]:
hit_ratio=hit/len(x_test)
hit_ratio #for k=10

0.274

In [36]:
encoded_imgs = encoder.predict(x_train)
train_restored = decoder.predict(encoded_imgs)

1276/1276 [==============================] - 2s 2ms/step


In [38]:
train_restored=pd.DataFrame(train_restored, columns=x_train.columns, index=x_train.index )

In [56]:
ndcg_cum=0
for user_id in x_train.index:
  gt_rank=get_rank(x_train.loc[user_id])
  pred_rank=get_rank(train_restored.loc[user_id])
  ndcg_cum+=ndcg_score(np.asarray([gt_rank]), np.asarray([pred_rank]), k=10)

In [55]:
ndcg_total=ndcg_cum/len(x_train)
ndcg_total #for k=5

0.5738225546693904

In [57]:
ndcg_total=ndcg_cum/len(x_train)
ndcg_total #for k=10

0.553943194365502

In [61]:
hit=0
k=10
for user_id in x_test.index:
  top_k_pred=list(restored.loc[user_id].sort_values(ascending=False).index[:k])
  #top_true=zero_one_gt.loc[user_id].sort_values(ascending=False).index[0] #이부분 코드 수정 !! (row별로 test idx 뽑아와서 소팅 없이)
  top_true=gt_matrix.loc[user_id,'test_joke_idx']
  if top_true in top_k_pred:
    hit+=1

In [59]:
hit_ratio=hit/len(x_test)
hit_ratio #for k=5

0.02925

In [62]:
hit_ratio=hit/len(x_test)
hit_ratio #for k=10

0.274